MSII_Base_Orbit_Determination.ipynb

Author: Elijah Sakamoto

Documentation: None



# MSII Base Orbit Determination

We want a fairly generic orbit template which we can modify for each spacecraft design, and test to find the best one. Our old geosynchronous approach from MSI does not work because it cannot easily achieve our resolution requirement while maintaining coverage.

This issue is demonstrated below:

In [ ]:
from Formula_Sheet import *
from MSIII_Components import *
from MSIII_useful_consts import *

ms1_orbit = MSI_ORBITS[0] # take the first MSI orbit, they are all almost identical

# find the resolution at apogee (where we flew over the points)
ra = ms1_orbit["a"] * (1 + ms1_orbit["e"])
ha = ra - R_EARTH

# assume we are using the most expensive imager, get resolution
res_a = get_payload_resolution(PAYLOADS[0], ha) 
print(f"MSI Orbit Resolution On Approach: {res_a} m")

MSI Orbit Resolution On Approach: 108.46703022385832 m


This is far from ideal. We need to be able to see stuff that is 4m big, not 108m big. Granted we are approaching at apogee, which is farthest off. To give it the benefit of the doubt, let's instead change it to a circular orbit, so it doesn't zoom past the points too fast, but still doesn't stay too far off:

In [14]:
new_e = 0

# find the resolution at apogee (where we flew over the points)
ra = ms1_orbit["a"] * (1 + new_e)
ha = ra - R_EARTH

# assume we are using the most expensive imager, get resolution
res_a = get_payload_resolution(PAYLOADS[0], ha) 
print(f"MSI Circular Orbit Resolution On Approach: {res_a} m")

MSI Circular Orbit Resolution On Approach: 54.691015113249996 m


Yeah, that doesn't work either. 54 > 4, so we wouldn't be able to see anything. Clearly, we need to find a different approach.

## But what do we mean by "template" ?

We want most of our COEs to be figured out before we start messing with payloads. Realistically, the only COEs that depend on payload are semimajor axis and eccentricity, because we want to fly over at a height where our sensor can see the points. If we pick a reasonable semimajor axis, we can even constrain it to just changing eccentricity to accomodate payloads, because with that can exert enough control on height on approach to see what we want to.

## What are our constraints for this?
We came up with several constraints for our orbital template:
- Approach: We want to approach from apogee, because otherwise we will zoom past it and spend so little time over target that we will have almost no coverage.
- Semimajor Axis: We want to keep the semimajor axis small enough that most cameras will be able to modify eccentricity so that apogee is in a vision range to meet our target resolution.
- Nice Period: We want our period to be a nice multiple of our 24 hour days. Otherwise, the position of the satellite on its ground track is different each time it comes around because the Earth has rotated under it. This is bad because it makes our math nasty and our coverage erratic or unpredictable.

### With these in mind, lets brainstorm + test some possible solutions!

In [ ]:
from MSIII_Constellation import *

# get the average latitude of the points of interest (useful for inclination)
avg_latitude = get_avg_latitude(POINTS_OF_INTEREST)
print(f"Avg. Latitude: {avg_latitude} deg")

# some ideas for what our period could be
ideas = [
    ["Geo-Sync Apogee", h_to_sec(24)], # we already ruled this one out, but a useful comparison
    ["12hr Apogee", h_to_sec(12)], # return to same point every 24 hours
    ["6hr Apogee", h_to_sec(6)],  # same here
    ["2hr Apogee", h_to_sec(2)]
]

# take a period idea, turn it into a basic constellation
def convert_period_idea_to_apogee_constellation(idea, idea_payload=PAYLOADS[0], idea_struct=STRUCTURES[2], idea_adcs=ADCS[2]) -> MSII_Constellation:
    idea_period = idea[1] # store the target period of the idea
    idea_max_h = get_payload_max_height(idea_payload, MIN_RESOLUTION) # how high can we approach from?
    
    idea_a = period_to_axis(idea_period) # calculate a from period (in seconds)
    idea_i = get_avg_latitude(POINTS_OF_INTEREST) # average out the inclinations of each point
    if (idea_i < 0):
        idea_i += 180 # allow for points south of the equator, should not be a problem
    idea_e = apogee_h_to_e(idea_a, idea_max_h, False) # what is our max eccentricity to get that height?
    idea_w = 270 # we want to set argument of perigee to 270 to simplify calculations
    if (get_avg_latitude(POINTS_OF_INTEREST) < 0):
        idea_w = 90 # allow for points south of the equator, should not be a problem
    
    idea_sat_coes = []
    for sat_num in range(NUM_SATS):
        time_on_target = (idea_period / NUM_SATS) * sat_num # stagger the times on target evenly
        
        sat = {}
        sat["a"] = idea_a # initialize all the variables that stay the same accross sats
        sat["e"] = idea_e
        sat["w"] = idea_w
        sat["i"] = idea_i
        
        # now for the hard bit, we need to change raan and v so they each hit apogee over
        # the target points at staggered times
        avg_long = get_avg_longitude(POINTS_OF_INTEREST)
        # the raan that would put us over the points in BORG at t=0
        avg_raan_t_0 = avg_long - INTIAL_EARTH_POS - idea_w
        # the raan that would put us over the points in BORG at time on target
        avg_raan_t = avg_raan_t_0 + ((360 / DAY_AS_SECONDS) * time_on_target) # assume 24 hour Earth period
        sat["raan"] = avg_raan_t # set that as our raan
        sat["v"] = calc_v_at_t0_to_hit_apogee_at_t(idea_a, idea_e, time_on_target)
        idea_sat_coes.append(sat) # add the satellite coes to our list
    
    assert(len(idea_sat_coes) == NUM_SATS) # make sure we have the right amount of sats
    # assert(idea_sat_coes["a"] * (1 + idea_sat_coes["e"]) < )
    
    idea_constellation_dict = {
        "structure": idea_struct, # use the best structure by default so it fits the camera for sure
        "adcs": idea_adcs, # best adcs by default
        "payload": idea_payload, # same with the imager
        "sats": idea_sat_coes
    }
    
    return MSII_Constellation(idea_constellation_dict) # wrap it all nicely in a constellation obj


# convert them into Constellation objects
for idea in ideas:
    idea[1] = convert_period_idea_to_apogee_constellation(idea)
    # now, the list for each idea should read [name, MSII_Constellation object]
    

    
# test them all
for idea in ideas:
    # write out the idea title real big
    print(f"\n\n================================ {idea[0]} ================================\n") 
    print(idea[1].to_string(idea[0]))
    print(idea[1].get_concise_summary().to_string(idea[0]))
    print(idea[1].assess_coverage().to_string(idea[0]))
    print(idea[1].assess_orbit_viability().to_string(idea[0]))

    
    



Avg. Latitude: 50.824999999999996 deg


================================ Geo-Sync Apogee ================================

Constellation Geo-Sync Apogee:
	Payload:   Astra Hi-Res Imager
	Structure: Option III
	ADCS:      Option III
	ORBIT: 
		a    : 42241.097730143825
		e    : 0.0001
		i    : 50.824999999999996
		raan : 110.375
		w    : 270
		v    : 180.0
	ORBIT: 
		a    : 42241.097730143825
		e    : 0.0001
		i    : 50.824999999999996
		raan : 230.375
		w    : 270
		v    : 60.00992454035004
	ORBIT: 
		a    : 42241.097730143825
		e    : 0.0001
		i    : 50.824999999999996
		raan : 350.375
		w    : 270
		v    : 60.00496220815088

Constellation Geo-Sync Apogee Concise Summary:
	Passes:       True
	In Budget:    True
	Viable Orbit: True
	Volume Fits:  True

Constellation Geo-Sync Apogee Point Coverage:
	Can Point Within SW:     True
	% Coverage:              0.0%
	Points Sampled:          4000
	Points in range and LOS: 0
	Points in range:         0
	Points in LOS:           2751
	Closest Ap

### Oops
It appears we have miscalculated. None of our satellites are getting in range of the points for any reasonable duration! I believe this is likely because we are trying to approach from apogee to get higher los coverage, at the expense of greater distance from the points. Let's run a quick test to confirm this is the case:

In [16]:
# compute the max R values for each of the payloads
for payload in PAYLOADS:
    print(str(get_payload_max_height(payload, MIN_RESOLUTION) + R_EARTH) + " km")
    

9001.087819672131 km
8782.508584699453 km
8222.399295081967 km
7783.289224824356 km


It is as we suspected, this does not work. The maximum R for each of the sensors is greater than a for even the 6hr orbit. This means that it is impossible to approach them from apogee at a sufficiently low altitude and spend much time over the points. We will have to think of something else.

### Something Else
Alright, I guess we will have to go with a perigee approach. The problems with this still remain -- first and foremost that we will have reduced time over the target because we will be moving faster. This, unfortunately, cannot be mitigated in any reasonable way.

One issue that we can and will need to mitigate, though, is that in perigee (unlike in apogee), we reach the closest point to Earth at the exact moment of perigee. This means that we cannot set the perigee altitude to the exact camera range, because then the single point at which the satellite will be in range is the exact moment of perigee. We want to be able to take more than just one photo per orbit, so we will have to set perigee altitude lower than camera range in hopes of being able to see the target from points around perigee, not just at perigee. This is not an exact science the way we implement it (we are just guessing altitude offsets), but a more precise answer could be developed by looping over many eccentricity values, testing them for coverage, and picking whichever one maximizes that while still adhering to the constraints. However, this drastically increases the number of coverage computations we need to make, each of which is very expensive. For this reason, we will not be doing that.

In [17]:
# we will test all of the same period ideas
ideas = [
    ["Geo-Sync Perigee", h_to_sec(24)],
    ["12hr Perigee", h_to_sec(12)], # return to same point every 24 hours
    ["6hr Perigee", h_to_sec(6)],  # same here
    ["2hr Perigee", h_to_sec(2)],
    ["1.7hr Perigee", h_to_sec(1.7)]
]



# take a period idea, turn it into a basic constellation
def convert_period_idea_to_perigee_constellation(idea, altitude_offset_margin=10, idea_payload=PAYLOADS[0], idea_struct=STRUCTURES[2], idea_adcs=ADCS[2]) -> MSII_Constellation:
    idea_period = idea[1] # store the target period of the idea
    idea_max_h = get_payload_max_height(idea_payload, MIN_RESOLUTION) # how high can we approach from?
    
    idea_a = period_to_axis(idea_period) # calculate a from period (in seconds)
    idea_i = get_avg_latitude(POINTS_OF_INTEREST) # average out the inclinations of each point
    if (idea_i < 0):
        idea_i += 180 # allow for points south of the equator, should not be a problem
    idea_e = perigee_h_to_e(idea_a, idea_max_h - altitude_offset_margin, False) # what is our max eccentricity to get that height?

    idea_w = 90 # we want to set argument of perigee to 90 to simplify calculations
    if (get_avg_latitude(POINTS_OF_INTEREST) < 0):
        idea_w = 270 # allow for points south of the equator, should not be a problem
    
    idea_sat_coes = []
    for sat_num in range(NUM_SATS):
        time_on_target = (idea_period / NUM_SATS) * sat_num # stagger the times on target evenly
        
        sat = {}
        sat["a"] = idea_a # initialize all the variables that stay the same accross sats
        sat["e"] = idea_e
        sat["w"] = idea_w
        sat["i"] = idea_i
        
        # now for the hard bit, we need to change raan and v so they each hit apogee over
        # the target points at staggered times
        avg_long = get_avg_longitude(POINTS_OF_INTEREST)
        # the raan that would put us over the points in BORG at t=0
        avg_raan_t_0 = avg_long - INTIAL_EARTH_POS - idea_w
        # the raan that would put us over the points in BORG at time on target
        avg_raan_t = avg_raan_t_0 + ((360 / DAY_AS_SECONDS) * time_on_target) # assume 24 hour Earth period
        sat["raan"] = avg_raan_t # set that as our raan
        sat["v"] = calc_v_at_t0_to_hit_perigee_at_t(idea_a, idea_e, time_on_target)
        idea_sat_coes.append(sat) # add the satellite coes to our list
    
    assert(len(idea_sat_coes) == NUM_SATS) # make sure we have the right amount of sats
    
    idea_constellation_dict = {
        "structure": idea_struct, # use the best structure by default so it fits the camera for sure
        "adcs": idea_adcs, # best adcs by default
        "payload": idea_payload, # same with the imager
        "sats": idea_sat_coes
    }
    
    return MSII_Constellation(idea_constellation_dict) # wrap it all nicely in a constellation obj

# convert them into constellations + test at the same time
for idea in ideas:
    # so it never gets noticeable drag
    altitude_offset = get_payload_max_height(PAYLOADS[0], MIN_RESOLUTION) - (MIN_DRAGLESS_ALTITUDE + 1) # +1 to prevent rounding errors
    idea[1] = convert_period_idea_to_perigee_constellation(idea, altitude_offset_margin=altitude_offset)
    # write out the idea title real big
    print(f"\n\n================================ {idea[0]} ================================\n") 
    print(idea[1].to_string(idea[0]))
    print(idea[1].get_concise_summary().to_string(idea[0]))
    print(idea[1].assess_coverage().to_string(idea[0]))
    print(idea[1].assess_orbit_viability().to_string(idea[0]))



================================ Geo-Sync Perigee ================================

Constellation Geo-Sync Perigee:
	Payload:   Astra Hi-Res Imager
	Structure: Option III
	ADCS:      Option III
	ORBIT: 
		a    : 42241.097730143825
		e    : 0.8347785125143755
		i    : 50.824999999999996
		raan : 290.375
		w    : 90
		v    : 0.0
	ORBIT: 
		a    : 42241.097730143825
		e    : 0.8347785125143755
		i    : 50.824999999999996
		raan : 50.375
		w    : 90
		v    : 146.44029712684926
	ORBIT: 
		a    : 42241.097730143825
		e    : 0.8347785125143755
		i    : 50.824999999999996
		raan : 170.375
		w    : 90
		v    : 169.65931160912885

Constellation Geo-Sync Perigee Concise Summary:
	Passes:       True
	In Budget:    True
	Viable Orbit: True
	Volume Fits:  True

Constellation Geo-Sync Perigee Point Coverage:
	Can Point Within SW:     True
	% Coverage:              0.1895%
	Points Sampled:          4000
	Points in range and LOS: 758
	Points in range:         758
	Points in LOS:           3438
	Close

### We have a winner!
Really, this is not a very good winner. With this winner, we only get coverage .18% of the time, which is not what we were hoping for. However, this includes at least 3 short daily viewing sessions each day, because all three satellites will fly over the points at times evenly spaced throughout the day. If the enemy is up to something, we will see it within at most (24/3=8) hours. 

Some might be wondering why we did not just use an orbit that stays at the target altitude for its entire orbit (circular). There is a very good reason for this. Were we to do this, we would get a period which is does not devide envenly into 24 hours. This means that, every day, the orbit ground track would shift just the smallest bit on the face of the Earth, because -- while the satellite is returning to the same coordinates -- it is doing so at a time when the Earth is rotated in a different position. It is certainly true that we would have comparable coverage for our first orbit, but the phase shift would gradually become worse and worse until we were looking at some random point in the middle of one of Earth's oceans.

Additionally, there is the concern that, by staying ~600 km (our mission altitude) in altitude for the entire lifetime of the satellite, we would experience much greater drag, lowering the lifespan. We could also experience far more significant wear and tear from colliding with oxygen in the upper reaches of the atmosphere. This is not good, so it is better to use this highly elliptical orbit.

#### See below for the details of our selected orbital template vv
We will modify this template in order to be suitably used for our further testing on equipment.

In [18]:
# display a comprehensive report on the winning template idea
print(f"\n\n================================ {ideas[0][0]} ================================\n") 
print(ideas[0][1].to_string(ideas[0][0]))
print(ideas[0][1].get_comprehensive_summary().to_string(ideas[0][0]))



================================ Geo-Sync Perigee ================================

Constellation Geo-Sync Perigee:
	Payload:   Astra Hi-Res Imager
	Structure: Option III
	ADCS:      Option III
	ORBIT: 
		a    : 42241.097730143825
		e    : 0.8347785125143755
		i    : 50.824999999999996
		raan : 290.375
		w    : 90
		v    : 0.0
	ORBIT: 
		a    : 42241.097730143825
		e    : 0.8347785125143755
		i    : 50.824999999999996
		raan : 50.375
		w    : 90
		v    : 146.44029712684926
	ORBIT: 
		a    : 42241.097730143825
		e    : 0.8347785125143755
		i    : 50.824999999999996
		raan : 170.375
		w    : 90
		v    : 169.65931160912885

Constellation Geo-Sync Perigee Comprehensive Summary: 
	Constellation Geo-Sync Perigee Concise Summary:
		Passes:       True
		In Budget:    True
		Viable Orbit: True
		Volume Fits:  True
	
	Constellation Geo-Sync Perigee Orbit Viability: VIABLE
		ORBIT 1
			CATASTROPHIC?      False
			DRAGLESS?          True
			CRASHLESS?         True
			Max Altitude:      71124.921